# Heuristics - CVRP

In [ ]:
from itertools import cycle

import numpy as np
import pandas as pd
from scipy.spatial.distance import pdist, squareform
import matplotlib.pyplot as plt
import matplotlib as mpl
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

import functions as f
import importlib

## Instantiate data

In [ ]:
dataset = pd.read_csv("./data/custom_19.csv", index_col=0)
coordinates = dataset.loc[:, ["x", "y"]]

In [ ]:
n_vehicles = 4
start_locations = [0, 2, 3, 5]
end_locations = [0, 2, 3, 5]

N = coordinates.shape[0]

In [ ]:
# assume travel time is proportional to euclidean distance
time_matrix = squareform(pdist(coordinates, metric="euclidean"))
time_matrix = np.round(time_matrix * 1e4, decimals=0).astype(int)

## Model

In [ ]:
# Create the routing index manager: number of nodes, number of vehicles, depot node
manager = pywrapcp.RoutingIndexManager(N, n_vehicles, start_locations, end_locations)

# Create Routing Model
routing = pywrapcp.RoutingModel(manager)

### Parameters

In [ ]:
# Same valid for any callback related to arcs/edges
def time_callback(from_index, to_index):
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return time_matrix[from_node, to_node]


transit_callback_index = routing.RegisterTransitCallback(time_callback)

In [ ]:
# # Same valid for any callback related to nodes
# def demand_callback(from_index):
#     from_node = manager.IndexToNode(from_index)
#     return demands[from_node]

# demand_callback_index = routing.RegisterUnaryTransitCallback(demand_callback)

### Constraints

In [ ]:
# # Any constraint associated with vehicles can take same arguments
# routing.AddDimensionWithVehicleCapacity(
#     demand_callback_index,
#     0,  # null capacity slack
#     [capacity,] * n_vehicles,  # vehicle maximum capacities (list for each vehicle)
#     True,  # start cumul to zero
#     'Capacity'
# )

### Objective

In [ ]:
# Define cost of each arc
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

### Solution

In [ ]:
# Setting heuristic strategies
search_parameters = pywrapcp.DefaultRoutingSearchParameters()

search_parameters.local_search_metaheuristic = (
    routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH
)

search_parameters.time_limit.FromSeconds(1)

# Solve the problem
solution = routing.SolveWithParameters(search_parameters)

In [ ]:
solution.ObjectiveValue()

### Plot Results

In [ ]:
importlib.reload(f)
tours = f.compile_tours(n_vehicles, routing, manager, solution)

In [ ]:
importlib.reload(f)
fig = f.plot_tours(tours, coordinates)
fig.show()